# Exercise 4: LMS algorithm for stock price prediction

## 4.1 Introduction

In this exercise the *trainlms* function given below will be used.

**Tasks**

1. What does the function *trainlms* do?

**Answer**

1. Function *trainlms* optimizes weights to have LMS error as small as possible?

In [ ]:
import numpy as np

def predict(W, A):
    return W@np.vstack([-np.ones((1, A.shape[1])), A])

def trainlms(ni, x, d, W, max_epoch):
    w=W.copy()
    
    n=0
    errors=[]
    while (n<max_epoch):
        n+=1
        y=predict(w, x)
        e=d-y
        w+=ni*e@np.vstack([-np.ones((1, x.shape[1])), x]).T
        error=np.sum(np.square(e))
        errors.append(error)
        if (error<0.02):
            break
    return w, errors

## 4.2 Stock price movement

In this experiment we will use the LMS algorithm in order to predict the stock price for a given stock. Load the data from file *stock.txt* into variable *data*. Numbers (elements of vector *data*) show the movement of average daily price of the given share in time. Plot the variable *data* using the following commands:

In [ ]:
data=np.loadtxt("sample_data/stocks.txt", delimiter=",")

import matplotlib.pyplot as plt
plt.plot(range(1, data.shape[0]+1), data)
plt.show()

The goal of this exercise is to use several (let us say $N$) previous share prices in order to predict the share price for today. This is useful because we can buy or sell the price before its price rises or falls and in that way increase our profit or lower our loss. The first step is to find the input-output pairs, which will be used on order to train our network. The size of this set will be defined using variable $i$. Our inputs will be vectors $\mathbf{a}_i$, which we put in matrix $\mathbf{A}$. Outputs are scalar values, which we put in vector $y$.

**Tasks**

1. Write the function *memory*, which for given day in the year (index of the variable *data*) constructs a column vector $\mathbf{a}$ whose elements represent the prices for last *N* days but without the price for today.
2. Using the function *memory *write a function *memorize*, which for given inputs (*data*, *day*, *N*, *i*) constructs the matrix $\mathbf{A}$ in which each column vector represents the $\mathbf{a}_i$ for the previous day. So, matrix $\mathbf{A}$ memorizes $i$ memories which we will use in order to train our network.

In [ ]:
# WRITE YOUR CODE HERE
def memory(data, day, N):
  return np.array([data[day-N-1:day-1]], dtype=np.float64).T

def memorize(data, day, N, i):
  if day < i + N - 1:
    print("day < i + N - 1")
  A = memory(data,day,N)
  for j in range(1,i):
    A = np.hstack((A,memory(data,day-j,N)))
  return A

In [ ]:
day=151
N=70
i=50
A=memorize(data, day, N, i)

Construct the matrix $\mathbf{A}$ using the command:

The output vector is constructed using the command:

In [ ]:
y=np.array([data[day-i+1:day+1]])

We initialize the perceptron as follows:

In [ ]:
def initp(data, labels):
    return -0.5+np.random.rand(labels.shape[0], data.shape[0]+1)

W=initp(A, y)

We train the perceptron using the command:

In [ ]:
ni=1e-8
max_num_iter=10000

W1, errors=trainlms(ni, A, y, W, max_num_iter)

Weights of the network are memorized in the matrix $\mathbf{W}_1$. Variable *ni* should be experimentally found. Train the network for different values of *i*, *N*, *max_num_iter*. We can visualize the output of the network on the training data using the following command:

In [ ]:
p=predict(W1, A)
plt.plot(range(1, y[0].shape[0]+1), y[0])
plt.plot(range(1, p[0].shape[0]+1), p[0], c="red")
plt.show()

In [ ]:
# WRITE YOUR CODE HERE
day=201
ni=0.6e-8
max_num_iter=10000
i_s = [100, 30, 50]
N_s = [80, 20]
Weights = []
table = [["i" , "N", "iterations", "profit"]]
for i in i_s:
  for N in N_s:
    table.append([i,N,max_num_iter])
    A=memorize(data, day, N, i)
    y=np.array([data[day-i+1:day+1]], dtype=np.float64)
    W=initp(A, y)
    W1, errors=trainlms(ni, A, y, W, max_num_iter)
    Weights.append(W1)
    p=predict(W1, A)
    plt.plot(range(1, y[0].shape[0]+1), y[0], label = "Real values")
    plt.plot(range(1, p[0].shape[0]+1), p[0], c="red", label= "Predicted")
    plt.legend(loc="best")
    plt.title("$i$ = " + str(i) + ", $N$ = " + str(N) + ", $iterations$ = " + str(max_num_iter))
    plt.show()

Here, the blue color shows the predicted values and the red color shows the real outputs.

**Tasks**

1. Memorize different weight matrices ($\mathbf{W}_1$, $\mathbf{W}_2$...) with different combinations of following parameters *i*=30, 50 or 100; *N*=20, 50 or 80; *max_num_iter*=10000, 50000 or 500000. (Choose 6 different combinations.) For selected parameter combinations show the obtained predictions with real values. Comment the obtained results.

If we do not use any intelligence for stock price prediction and we assume the price will be (almost) the same tomorrow, the error we make is:

In [ ]:
a=data[day:day+1]          # price today - we assume the same price tomorrow
y=data[day+1:day+2]        # the real price tomorrow
err_oo=np.sum(np.abs(y-a)) # error

If we use the network trained on previous days to predict the price for tomorrow, the error is as follows:

In [ ]:
p=predict(W1, memorize(data, day+1, N, 1))
err_nn=np.sum(np.abs(y-p))

If we trade each day with the given stock, our error can be easily measured. Our potential profit can be calculated as follows:

In [ ]:
profit=err_oo-err_nn
print(profit)

To predict the price of the day after tomorrow, we could use the same network or we could train the network from scratch by also using the data of the day before the day after tomorrow i.e. by using the data of tomorrow. Which way would be better?

**Tasks**

1. Create a table in which you will show the obtained profit levels depending on the parameters used (*i*, *N*, and *max_num_iter*). For of these parameter triplets the profit should be the sum of the profit for every possible *day* for which training data defined by *i* and *N* exists.

In [ ]:
from tabulate import tabulate

In [ ]:
# WRITE YOUR CODE HERE
table = [["i" , "N", "iterations", "profit"]]
iter = 0
for idx,i in enumerate(i_s):
  for jdx,N in enumerate(N_s):
    profit = 0
    for j in range(0,i):
      a=data[day-j-2:day-j-1]          # price today - we assume the same price tomorrow
      y=data[day-j-1:day-j]        # the real price tomorrow
      err_oo=np.sum(np.abs(y-a))
      p=predict(Weights[iter], memorize(data, day-j+1, N, 1))
      err_nn=np.sum(np.abs(y-p))

      profit += err_oo-err_nn

    table.append([i,N,max_num_iter,profit])
    iter += 1  

print(tabulate(table, headers='firstrow'))